# How to connect Jupyter notebook (local instance) with Amazon S3

## AWS Credentials 

Create your access key:
- go to aws console
- IAM -> Users -> <User> -> Security credentials -> Download file (Store the keys in a secure location)

Install the AWS Command Line Interface on macOS:

$ pip install awscli --upgrade --user


$ aws configure

AWS Access Key ID [None]: [Access-key-from-csv-file]

AWS Secret Access Key [None]: [Secret-Access-key-from-csv-file]

Default region name [None]: [Enter]

Default output format [None]: [Enter]

Your credentials should be saved in ~/.aws/credentials file

## Connect to S3 in Jupyter Notebook  

In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = "--packages=org.apache.hadoop:hadoop-aws:2.7.3 pyspark-shell"

In [3]:
import configparser
import os
config = configparser.ConfigParser()
config.read(os.path.expanduser("~/.aws/credentials"))
access_id = config.get('default', "aws_access_key_id") 
access_key = config.get('default', "aws_secret_access_key")

In [4]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext('local')
spark = SparkSession(sc)

hadoop_conf=sc._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3n.impl", "org.apache.hadoop.fs.s3native.NativeS3FileSystem")
hadoop_conf.set("fs.s3n.awsAccessKeyId", access_id)
hadoop_conf.set("fs.s3n.awsSecretAccessKey", access_key)

In [5]:
df=spark.read.orc("s3n://miquido-ds-test-backet/events_processing/events_processing_df_002_orc")
df.limit(5).toPandas().head()

,meta_id,meta_type,meta_timestamp,meta_trackingId,meta_appInstanceId,meta_ipAddress,data_appUserId,data_data_input_id,data_fieldToCopy,data_device_device_type,...,meta_trackingId_filled,meta_appInstanceId_filled,data_appUserId_filled,data_data_input_id_filled,data_device_device_type_filled,data_device_device_model_filled,meta_type_filled,data_playlist_filled,data_fieldToCopy_filled,meta_id_filled
0,333A,user.signedIn,100000000,klassikradio-dev,a100,00.000.000.000,asgjsskKi81217@test1234.com,1,1234.0,mobile,...,klassikradio-dev,a100,asgjsskKi81217@test1234.com,1,mobile,M8,user.signedIn,None,1234,333A
1,333A,user.signedIn,100000020,klassikradio-dev,a100,00.000.000.000,asgjsskKi81217@test1234.com,2,NaN,None,...,klassikradio-dev,a100,asgjsskKi81217@test1234.com,2,mobile,M8,user.signedIn,None,1234,333A
2,333A,user.signedIn,100000040,klassikradio-dev,a100,00.000.000.000,asgjsskKi81217@test1234.com,3,NaN,None,...,klassikradio-dev,a100,asgjsskKi81217@test1234.com,3,mobile,M8,user.signedIn,None,1234,333A
3,333A,user.signedIn,100000060,klassikradio-dev,a100,00.000.000.000,asgjsskKi81217@test1234.com,4,3562.0,mobile,...,klassikradio-dev,a100,asgjsskKi81217@test1234.com,4,mobile,H45,user.signedIn,None,3562,333A
4,333A,user.signedIn,100000080,klassikradio-dev,a100,00.000.000.000,asgjsskKi81217@test1234.com,5,NaN,None,...,klassikradio-dev,a100,asgjsskKi81217@test1234.com,5,mobile,H45,user.signedIn,None,3562,333A
